# Подготовка данных

Мы использользуем набор данных PadChest 

TODO: подготовить больше текста/описания для слайдов, см. https://bimcv.cipf.es/bimcv-projects/padchest/

In [2]:
!mkdir -p /content/padchest

# download sample images from padchest
!gdown 1FJ8T2lYCtTxtfhatpcXBpXvEBx9gTf81
!unzip -qo /content/sample.zip -d /content/padchest/
# download sample 2 images
!gdown 1_mKm1hlqAbxDfvZRKAJjW4icZPqAXICD
!unzip -qo /content/padchest_sample2.zip -d /content/padchest/

# labels from ML drive
!gdown 1WYuYC-p77joqAv7TDivS3zGMD3NqTk4T
!mv /content/PADCHEST_chest_x_ray_images_labels_160K_01.02.19.csv /content/padchest/

Downloading...
From: https://drive.google.com/uc?id=1FJ8T2lYCtTxtfhatpcXBpXvEBx9gTf81
To: /content/sample.zip
100% 175M/175M [00:00<00:00, 252MB/s]
Downloading...
From: https://drive.google.com/uc?id=1_mKm1hlqAbxDfvZRKAJjW4icZPqAXICD
To: /content/padchest_sample2.zip
100% 1.20G/1.20G [00:04<00:00, 251MB/s]
Downloading...
From: https://drive.google.com/uc?id=1WYuYC-p77joqAv7TDivS3zGMD3NqTk4T
To: /content/PADCHEST_chest_x_ray_images_labels_160K_01.02.19.csv
100% 99.2M/99.2M [00:00<00:00, 247MB/s]


Install pytorch X-ray library

In [3]:
!pip install torchxrayvision

     |████████████████████████████████| 29.0 MB 1.5 MB/s 


In [4]:
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import torch  
import torchvision
import torchxrayvision as xrv
import pylab
import torch.optim as optim
import torch.nn as nn
from pathlib import Path
from tqdm import tqdm

In [5]:
PATH = Path('/content/padchest')

In [6]:
transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop(),xrv.datasets.XRayResizer(224)])
dataset = xrv.datasets.PC_Dataset(imgpath=PATH,csvpath=PATH/'PADCHEST_chest_x_ray_images_labels_160K_01.02.19.csv', transform=transform)

In [7]:
print(dataset)

{'Air Trapping': {0.0: 59407, 1.0: 2285},
 'Aortic Atheromatosis': {0.0: 60419, 1.0: 1273},
 'Aortic Elongation': {0.0: 56611, 1.0: 5081},
 'Atelectasis': {0.0: 59273, 1.0: 2419},
 'Bronchiectasis': {0.0: 60821, 1.0: 871},
 'Cardiomegaly': {0.0: 56305, 1.0: 5387},
 'Consolidation': {0.0: 61217, 1.0: 475},
 'Costophrenic Angle Blunting': {0.0: 59587, 1.0: 2105},
 'Edema': {0.0: 61584, 1.0: 108},
 'Effusion': {0.0: 60067, 1.0: 1625},
 'Emphysema': {0.0: 61146, 1.0: 546},
 'Fibrosis': {0.0: 61351, 1.0: 341},
 'Flattened Diaphragm': {0.0: 61379, 1.0: 313},
 'Fracture': {0.0: 60030, 1.0: 1662},
 'Granuloma': {0.0: 60135, 1.0: 1557},
 'Hemidiaphragm Elevation': {0.0: 60806, 1.0: 886},
 'Hernia': {0.0: 60704, 1.0: 988},
 'Hilar Enlargement': {0.0: 58875, 1.0: 2817},
 'Infiltration': {0.0: 57383, 1.0: 4309},
 'Mass': {0.0: 61186, 1.0: 506},
 'Nodule': {0.0: 59502, 1.0: 2190},
 'Pleural_Thickening': {0.0: 59617, 1.0: 2075},
 'Pneumonia': {0.0: 59782, 1.0: 1910},
 'Pneumothorax': {0.0: 61595, 1.

In [ ]:
len_dataset=len(dataset)
n_train=int(0.9*len_dataset)
n_test=int(0.1*len_dataset)+1
print(f'Total samples: {len_dataset}, train size size: {n_train}, test set size: {n_test}')

In [ ]:
train_ds, test_ds = torch.utils.data.random_split(dataset, [n_train,n_test])
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=4,shuffle=True, num_workers=4)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=4,shuffle=True, num_workers=1)

In [ ]:
model = xrv.models.DenseNet(num_classes=2).cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

Features and target columns

In [ ]:
features = 'PA' # 
target = 'lab'  #

Training loop

In [ ]:
train_loss_history, test_loss_history = [], []
for epoch in tqdm(range(10)):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dl, 0):
        inputs=data[features].cuda()
        labels=data[target].long().cuda()
        labels=labels[:,2]
       
        # get the inputs; data is a list of [inputs, labels]
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    test_loss=0.0

    for i, data in enumerate(test_dl, 0):
        inputs=data[features].cuda()
        labels=data[target].long().cuda()
        labels=labels[:,2]

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # print statistics
        test_loss += loss.item()
        
    train_loss_history.append(running_loss)
    test_loss_history.append(test_loss)

In [ ]:
plt.plot(train_loss_history, label='trainig loss')
plt.plot(test_loss_history, label='testing loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

to be continued.